In [ ]:
%matplotlib inline

import pelops.datasets.slice as slice
import pelops.features.hog as hog
import pelops.datasets.featuredataset as feature_dataset
import pelops.experiment_api.experiment as exp
import pelops.analysis.analysis as analysis
import matplotlib.pyplot as plt

In [ ]:
# Load SLiCE chips and generate HOG features

DATA_DIR = '/sk/data/STR-multiCamChips-20170322-1900'
HDF_FILE = '/sk/data/slice_hog_test2.hdf5'

data_set = slice.SliceDataset(DATA_DIR, debug=True)
hog_features = hog.HOGFeatureProducer(data_set)
feature_dataset.FeatureDataset.save(HDF_FILE, *hog_features.return_features())

In [ ]:
# Run various experiments to generate CMC

ITEMS_PER_CAMERA = 10
Y_RANDOM = 1024
CAMERAS = 2
DROPPED = 0
CMC_CNT = 500
EXPERIMENTS = 500

slice_hog_features = feature_dataset.FeatureDataset(HDF_FILE)
experiment_gen = exp.ExperimentGenerator(slice_hog_features, CAMERAS, ITEMS_PER_CAMERA, DROPPED, Y_RANDOM)
experiment_hldr = analysis.repeat_pre_cmc(slice_hog_features, experiment_gen, NUMCMC=CMC_CNT, EXPPERCMC=EXPERIMENTS)
stats, gdata = analysis.make_cmc_stats(experiment_hldr, ITEMS_PER_CAMERA)

In [ ]:
# Plot experiment results

figure = plt.figure()
ax = plt.subplot(111)
ax.plot(gdata.transpose())
plt.title('{} CMC curves with {} experiments per curve'.format(CMC_CNT, EXPERIMENTS))
ax.legend(('-stddev','avg','+stddev'), bbox_to_anchor=(1, -0.05), fancybox=True, shadow=True, ncol=5)
plt.show()